# Fireveg DB - download and explore Austrait data

Author: [José R. Ferrer-Paris](https://github.com/jrfep) 

Date: July 2024

This Jupyter Notebook includes [Python](https://www.python.org) code to download and explore data from [AusTraits](https://austraits.org/).

> Falster, Gallagher et al (2021) AusTraits, a curated plant trait database for the Australian flora. Scientific Data 8: 254, <https://doi.org/10.1038/s41597-021-01006-6>

## Set-up

### Import modules

Let's start loading the libraries

In [1]:
from pathlib import Path
import os
import sys
import json
import urllib
from zipfile import ZipFile
import pandas as pd
import numpy as np
from pybtex.database.input import bibtex
import yaml
# Pyprojroot for easier handling of working directory
import pyprojroot

### Define paths for input and output

Define project directory using the `pyprojroot` functions, and add this to the execution path.

In [2]:
repodir = pyprojroot.find_root(pyprojroot.has_dir(".git"))
sys.path.append(str(repodir))

Path to the folder to downloaded data:

In [3]:
outputdir = repodir / "data/austraits/"

if not os.path.isdir(outputdir):
    os.makedirs(outputdir)

### Load own functions

Load functions from lib folder, we will use a function to read db credentials, one for executing database queries and three functions for extracting data from the reference description string

In [4]:
import lib.austraits_util as aust

## Download _austraits_ data 
We will download the file from the [Zenodo repository](https://zenodo.org/record/5112001) using the API url and saving this under the data folder.

In [5]:
dataset_parent = "https://zenodo.org/api/records/3568417" # parent record
dataset = "https://zenodo.org/api/records/5112001" # older version: 3.0.2


We use urllib to open the url and read the data (if successfully connected!)

In [6]:
def getResponse(url):
    operUrl = urllib.request.urlopen(url)
    if(operUrl.getcode()==200):
       data = operUrl.read()
    else:
       print("Error receiving data", operUrl.getcode())
    return data
zrecord = getResponse(dataset)

Response data is in json format, need to parse it:

In [7]:
jsonData = json.loads(zrecord)

In [8]:
jsonData.keys()

dict_keys(['created', 'modified', 'id', 'conceptrecid', 'doi', 'conceptdoi', 'doi_url', 'metadata', 'title', 'links', 'updated', 'recid', 'revision', 'files', 'owners', 'status', 'stats', 'state', 'submitted'])

The json data includes a list of files:

In [9]:
for files in jsonData['files']:
    print(files['key'])


dictionary.html
austraits-3.0.2.zip
readme.txt
NEWS.md
austraits-3.0.2.rds


We want to download the zip file with the csv_files

In [10]:
outputfile = outputdir / jsonData['files'][1]['key']

if os.path.isfile(outputfile):
    print('File exists')
else:
    resp = getResponse(jsonData['files'][1]['links']['self'])
    output = open(outputfile,'wb')
    output.write(resp)
    output.close()

File exists


## Explore data from one version of Austraits

What versions are available in our local folder?

In [11]:
os.listdir(outputdir)

['austraits-6.0.0.zip', 'austraits-3.0.2.zip']

We used the version 3.0.2 before, let's try version 6.0.0.
We will read from one of these zipfiles the data that we need:

In [12]:
austversion='austraits-6.0.0'
zfobj = ZipFile(outputdir / (austversion+".zip") ) 
zfobj.namelist()

['austraits-6.0.0/',
 'austraits-6.0.0/schema.yml',
 'austraits-6.0.0/taxa.csv',
 'austraits-6.0.0/methods.csv',
 'austraits-6.0.0/definitions.yml',
 'austraits-6.0.0/build_info.md',
 'austraits-6.0.0/contributors.csv',
 'austraits-6.0.0/contexts.csv',
 'austraits-6.0.0/excluded_data.csv',
 'austraits-6.0.0/locations.csv',
 'austraits-6.0.0/traits.csv',
 'austraits-6.0.0/taxonomic_updates.csv',
 'austraits-6.0.0/metadata.yml',
 'austraits-6.0.0/sources.bib']

We will need to read the files with the definitions (in _yaml_ format), the sources or references (in _bibtex_ format) and the traits and taxonomic data (in _csv_ format)

#### Read dictionary in yml format


In [13]:
with zfobj.open(austversion+'/definitions.yml') as file:
    try:
        ATdefinitions = yaml.safe_load(file)   
    except yaml.YAMLError as exc:
        print(exc)

In [14]:
ATdefinitions['resprouting_capacity'].keys()

dict_keys(['label', 'description', 'comments', 'type', 'allowed_values_levels', 'entity_URI'])

#### Read references in bibtex format

Here we parse the bibliography file, and check functions to extract reference info and reference label.

In [15]:
parser = bibtex.Parser()
ATrefs = parser.parse_bytes(zfobj.open(austversion+'/sources.bib').read())

In [16]:
print(aust.extract_refinfo(ATrefs,'NSWFRD_2014'))
print(aust.extract_reflabel(ATrefs,'NSWFRD_2014'))

Kenny, Belinda; Orscheg, Corinna; Tasker, Elizabeth; Gill, Malcolm A.; Bradstock, Ross (2014) {NSW Flora Fire Response Database, v2.1}
Kenny Orscheg Tasker Gill Bradstock 2014


#### Read traits in CSV format

In [17]:
ATtraits = pd.read_csv(zfobj.open(austversion+'/traits.csv'),
                       low_memory=False,
                    encoding="ISO-8859-1")
#                      encoding='cp1252')

#### Taxon information
Read from the `taxa.csv` file and query the APNI record in json format.

In [18]:
ATtaxa = pd.read_csv(zfobj.open( austversion+'/taxa.csv'))

Check information from one species

## Explore fire response traits from Austraits

In [19]:
alltraits = ATtraits['trait_name'].unique()
len(alltraits)

497

In [20]:
for item in alltraits:
    y = item.find("dorm")
    if y>-1:
        print(item)

seed_dormancy_class


In [21]:
### Dormancy type

In [22]:
aust.trait_summary(ATdefinitions, ATtraits, 'seed_dormancy_class')

**Description**: Physical [PATO:0002079], physiological [NCIT:C16990], and morphological [PATO:0000051] mechanisms [NCIT:C25343] that inhibit mature [PATO:0001701] seeds [PO:0009010] from germinating [GO:0009845].;Mechanisms that inhibit mature seeds from germinating and the environmental or experimental cues that allow a seed to break dormancy and germinate.

**Records in Austraits**: 4171.

**Allowed values**:

*morphological_dormancy*:
Seeds exhibit morphological dormancy, generally because an underdeveloped embryo needs to grow.


*morphophysiological_dormancy*:
Seeds exhibit morphophysiological dormancy, with both morphological and physiological dormancy mechanisms that must be overcome.


*non_dormant*:
Seeds are non-dormant.


*physical_dormancy*:
Seeds exhibit physical dormancy, due to one or more water-impermeable layers in the seed coat.


*physiological_dormancy*:
Seeds exhibit physiological dormancy, whereby chemical changes are required within the seed to break dormancy, including temperature stratification and after-ripening.


Frequency of values in this version of Austraits:


value
physical_dormancy                     1409
physiological_dormancy                1347
non_dormant                            956
morphophysiological_dormancy           456
non_dormant physiological_dormancy       3
Name: count, dtype: int64


In [23]:
### Dispersers

In [24]:
aust.trait_summary(ATdefinitions, ATtraits, 'dispersers')

**Description**: The taxa [NCIT:C40098] (biotic vectors) and abiotic mechanisms [NCIT:C25343] that disperse [OMIT: 0027297] seeds [PO:0009010] and fruit [PO:0009001] away from a parent plant.;Dominant diaspore dispersal agents, including both biotic agents (specific taxa) or abiotic mechanisms responsible for dispersing seeds (or other dispersal units) away from the parent plant.

**Records in Austraits**: 13652.

**Allowed values**:

*abiotic*:
Fruits are dispersed by unspecified abiotic mechanisms.


*animals*:
Animals are an important fruit dispersal agent.


*ants*:
Ants are an important fruit dispersal agent.


*bats*:
Bats are an important fruit dispersal agent.


...

*water*:
Water is an important fruit dispersal agent.


*water_currents*:
Water currents are an important fruit dispersal agent; the diaspore may be either floating or submerged in fresh/saltwater currents.


*wind*:
Wind is an important fruit dispersal agent.


Frequency of values in this version of Austraits:


value
wind                                      4109
invertebrates                             2502
birds                                     1959
mammals                                   1008
vertebrates                                564
                                          ... 
vertebrates water wind                       1
garden_refuse                                1
abiotic animals birds vertebrates wind       1
birds passive water                          1
mammals water                                1
Name: count, Length: 75, dtype: int64


In [25]:
### Dispersal appendage

In [26]:
aust.trait_summary(ATdefinitions, ATtraits, 'dispersal_appendage')

**Description**: A seed morphology trait [TO:0002629] which indicates external [NCIT:C44280] appendages [NCIT:C61460] on the seed [PO:0009010] or the seed's dispersal [OMIT: 0027297] structures [PATO:0000141] whose presumed function [NCIT:C25518] is to facilitate seed dispersal [EnvThes:21042].;External appendages of the dispersule presumed to facilitate seed dispersal away from the parent plant.

**Records in Austraits**: 3417.

**Allowed values**:

*aril*:
Fleshy outgrowth of a seed, that often attracts animals like birds or ants.


*bracts_and_glumes*:
When the bracts below the inflorescence are persistent and functional as dispersal appendages. This includes persistent glumes, the modified membranous bracts in the family Poaceae that surround the spikelet of a grass.


*bristles*:
Bristle-like projections. Trait value includes awns, which are slender, bristle-like projections in some grasses.


*elaiosome*:
Fleshy (often fatty) appendage on seeds that attracts ants.


...

*tumbleweed*:
Plant growth form, whereby the aboveground plant component detaches from the roots and is readily rolled by wind across the ground, aiding in dispersal.


*wing_or_plume*:
Broad term that includes a suite of appendages that aid in wind dispersal; highest level of resolution available for some studies.


*wings*:
Referring to wing-like seed extensions that aid in wind dispersal.


Frequency of values in this version of Austraits:


value
none                       1427
aril                        968
wings                       250
elaiosome                   208
pappus                      186
bristles                     93
fleshy_reward                58
hooks                        40
wing_or_plume                39
sarcotesta                   27
hairs                        24
plumose                      24
floral_parts                 15
spines                       10
inflated_parts               10
floating_seed                10
bracts_and_glumes             9
receptacle                    8
bristles hairs                3
hairs spines                  2
flotation_scales              2
bracts_and_glumes hairs       1
hairs hooks                   1
bristles floral_parts         1
tumbleweed                    1
Name: count, dtype: int64


In [27]:
### Dispersal syndrome

In [28]:
aust.trait_summary(ATdefinitions, ATtraits, 'dispersal_syndrome')

**Description**: A classification [NCIT:C25161] based on a taxon's [NCIT:C40098] seed morphology traits [TO:0000184] or fruit morphology traits [TO:0002629] that indicates the taxa (biotic vectors) or abiotic mechanisms [NCIT:C25343] that most likely disperse [OMIT: 0027297] seeds [PO:0009010] and fruit [PO:0009001] away from the parent plant.;Plant dispersal syndromes are generally assigned to species based on suites of diaspore characteristics associated with specific mechanisms of dispersal. Trait values are mostly given as the formal dispersal syndromes ending in 'chory', a suffix meaning 'a specified method of plant dispersal'.

**Records in Austraits**: 17522.

**Allowed values**:

*anemochory*:
Diaspore is dispersed by wind. (Synonyms, wind)


*anthrochory*:
Diaspore is dispersed by humans, either intentionally or unintentionally.


*atelochory*:
Diaspore dispersal is prevented. (Synonyms, antitelochory)


*autochory*:
Diaspore is dispersed by methods originating from the parent plant or diaspore.


...

*myrmecochory*:
Diaspores have elaiosomes (specialised nutritious appendages) that make them attractive for capture, transport and use by ants or related insects.


*undefined*:
Dispersal mechanism unknown. (Synonyms, unknown)


*zoochory*:
Diaspore is dispersed by animals, by an undescribed mechanism.


Frequency of values in this version of Austraits:


value
anemochory                                      5210
zoochory                                        3335
barochory                                       2256
epizoochory                                     2154
endozoochory                                    1531
myrmecochory                                     760
hydrochory                                       534
barochory myrmecochory                           408
undefined                                        182
barochory endozoochory myrmecochory              174
anemochory barochory                             173
ballistic                                        125
barochory zoochory                               104
ballistic myrmecochory                            94
anemochory hydrochory                             41
anemochory hydrochory zoochory                    35
barochory myrmecochory zoochory                   34
epizoochory myrmecochory                          29
anemochory ballistic                    

### Resprouting capacity

The Austrait variable used to be called 'fire_response' in version 3.0.2, and is now called 'resprouting_capacity' in version 6.0.0 

In [29]:
aust.trait_summary(ATdefinitions, ATtraits, 'resprouting_capacity')

**Description**: A fire response trait which indicates whether mature [PATO:0001701] plants [PO:0000003] from a taxon [NCIT:C40098] or population [PCO:0000001] are killed or have vegetative meristematic tissue [PO:0006079] (buds [PO:0000055]) that resprout [Cerrado:fire#PlantResprout] following a fire [ENVO:01000786] that causes 100% leaf scorch, with fire severity index [EnvThes:22120] an important contextual variable.;Trait indicating whether mature individuals of a taxon are killed, resprout, or have a mixed survival response when exposed to fire that causes 100% leaf scorch.  Species where at least 70% of plants resprout are designated `resprouters`, while those where fewer than 30% of plants resprout are designated as `fire killed`. Species with an intermediate response have a mixed fire response, and are coded as `fire_killed resprouts`.

**Records in Austraits**: 29896.

**Allowed values**:

*fire_killed*:
Fewer than 30% of plants resprout following a fire with 100% leaf scorch. (Synonyms, fire-sensitive)


*partial_resprouting*:
Between 30-70% of plants resprout following a fire with 100% leaf scorch.


*resprouts*:
More than 70% of plants resprout following a fire with 100% leaf scorch.


Frequency of values in this version of Austraits:


value
resprouts                                    16951
fire_killed                                  11132
partial_resprouting                           1500
fire_killed resprouts                          233
fire_killed partial_resprouting                 47
partial_resprouting resprouts                   30
fire_killed partial_resprouting resprouts        3
Name: count, dtype: int64


We used these rules to transform values from Austraits (v3.0.2) into the resprouting vocabulary

In [30]:
switcher={
        "fire_killed": "None",
        'not_fire_killed_does_not_resprout': "None",
        'fire_not_relevant': 'Unknown', 
        'fire_killed resprouts': 'Half',
        'unknown': 'Unknown',
        "resprouts": "All"
    }

We are using these rules to transform values from Austraits (v6.0.0) into the resprouting vocabulary

In [31]:
switcher={
        "fire_killed": "None",
        'fire_killed resprouts': 'Half',
        "resprouts": "All",
        "partial_resprouting": "Half",
        "fire_killed partial_resprouting": "Half",
        "partial_resprouting resprouts": "Half",
        "fire_killed partial_resprouting resprouts": "Half"
    }

Some examples of the records for this trait:

In [32]:
ss = (ATtraits['trait_name']=='resprouting_capacity')
target = ATtraits[ss].head()
target

,dataset_id,taxon_name,observation_id,trait_name,value,unit,entity_type,value_type,basis_of_value,replicates,...,source_id,location_id,entity_context_id,plot_context_id,treatment_context_id,collection_date,measurement_remarks,method_id,method_context_id,original_name
256311,Auld_2000,Anisopogon avenaceus,1,resprouting_capacity,resprouts,NaN,population,mode,expert_score,NaN,...,NaN,6.0,NaN,NaN,NaN,1990/1992,NaN,1.0,NaN,Anisopogon avenaceus
256314,Auld_2000,Asterolasia elegans,2,resprouting_capacity,fire_killed,NaN,population,mode,expert_score,NaN,...,NaN,7.0,NaN,NaN,NaN,1990/1992,NaN,1.0,NaN,Asterolasia elegans
256318,Auld_2000,Comesperma ericinum,3,resprouting_capacity,fire_killed,NaN,population,mode,expert_score,NaN,...,NaN,5.0,NaN,NaN,NaN,1990/1992,NaN,1.0,NaN,Comesperma ericinum
256322,Auld_2000,Conospermum taxifolium,4,resprouting_capacity,fire_killed,NaN,population,mode,expert_score,NaN,...,NaN,6.0,NaN,NaN,NaN,1990/1992,NaN,1.0,NaN,Conospermum taxifolium
256326,Auld_2000,Darwinia biflora,5,resprouting_capacity,fire_killed,NaN,population,mode,expert_score,NaN,...,NaN,1.0,NaN,NaN,NaN,1990/1992,NaN,1.0,NaN,Darwinia biflora


This is how one record looks like:

In [33]:
target.iloc[1]

dataset_id                           Auld_2000
taxon_name                 Asterolasia elegans
observation_id                               2
trait_name                resprouting_capacity
value                              fire_killed
unit                                       NaN
entity_type                         population
value_type                                mode
basis_of_value                    expert_score
replicates                                 NaN
basis_of_record               field_experiment
life_stage                               adult
population_id                               04
individual_id                              NaN
repeat_measurements_id                     NaN
temporal_context_id                        NaN
source_id                                  NaN
location_id                                7.0
entity_context_id                          NaN
plot_context_id                            NaN
treatment_context_id                       NaN
collection_da

And we can transform this into a record for the fireveg database using a custom function:

In [34]:
BIONET = pd.read_excel(repodir/ "data" / "vis-survey-datasheet-6000.PowerQuery.20210708.xlsx")
target.fillna("nan",inplace=True)
reflist=list()
records=list()
for idx, row in target.iterrows():
    record=aust.create_record(row, ATrefs, vocab=switcher, taxlist=BIONET)
    refid=row['dataset_id']
    if refid not in reflist:
        reflist.append(refid)
    refid=row['source_id']
    if refid not in reflist:
        reflist.append(refid)
    records.append(record)
records[0]

{'main_source': 'austraits-6.0.0',
 'additional_notes': ['Values reclassified by JRFP',
  'Automatic extraction with python script'],
 'raw_value': ['resprouting_capacity', 'resprouts', 'mode'],
 'original_notes': ['observation_id', '1', 'location id:', '6.0'],
 'original_sources': ['Auld Keith Bradstock 2000'],
 'species': 'Anisopogon avenaceus',
 'species_code': '4749',
 'weight': 1,
 'weight_notes': ['python-script import', 'default of 1'],
 'norm_value': 'All'}

### Post fire flowering

In [35]:
aust.trait_summary(ATdefinitions, ATtraits, 'post_fire_flowering')

**Description**: A fire response trait which is a taxon's [NCIT:C40098] flowering [PO:0007016] response [NCIT:C25755] to fire [ENVO:01000786], indicating if the magnitude [NCIT_C49158] of flowers [PO:0009046] increases, decreases or remains unchanged following a fire.;Indication of whether the magnitude of flowers produced by a taxon increases, decreases or remains unchanged following a fire.

**Records in Austraits**: 431.

**Allowed values**:

*fire_dependent_flowering*:
Plants predominantly flower after a fire, usually within the first 1-4 growing seasons following the fire.


*fire_enhanced_flowering*:
Flowering is enhanced after a fire, with plants flowering more prolifically for the first 1-4 growing seasons following the fire.


*fire_independent_flowering*:
Flowering is independent of fire, with the magnitude of flowers produced by plants unchanged following a fire.


*fire_suppressed_flowering*:
Flowering is suppressed or absent after a fire, with plants producing fewer flowers for at least the first 1-4 growing seasons following the fire.


Frequency of values in this version of Austraits:


value
fire_enhanced_flowering                                314
fire_independent_flowering                              68
fire_dependent_flowering                                40
fire_suppressed_flowering                                6
fire_dependent_flowering fire_independent_flowering      1
fire_dependent_flowering fire_enhanced_flowering         1
fire_enhanced_flowering fire_suppressed_flowering        1
Name: count, dtype: int64


### Seedbank location

In [36]:
aust.trait_summary(ATdefinitions, ATtraits, 'seedbank_location')

**Description**: The location [NCIT:C25341] of a plant's [PO:0000003] seedbank [MESH:D000068098], indicating locations where mature [PATO:0001701] seeds [PO:0009010] are stored and remain viable [PATO:0000169] until germination [GO:0009845].;Location of a plant's seedbank, the location where mature seeds are stored and viable until germinating.

**Records in Austraits**: 4164.

**Allowed values**:

*canopy_seedbank*:
Seeds stored in the canopy, usually still within a woody cone.


*canopy_seedbank_absent*:
Seed explicitly not stored in the canopy.


*none*:
Seeds are not stored.


*seedbank*:
A seedbank exists, but its locations is not specified.


*soil_seedbank*:
Seeds stored in the soil.


*soil_seedbank_absent*:
Seed explicitly not stored in the soil.


Frequency of values in this version of Austraits:


value
soil_seedbank                                  2525
canopy_seedbank                                 696
canopy_seedbank_absent soil_seedbank            369
canopy_seedbank_absent                          210
canopy_seedbank soil_seedbank_absent            175
none                                            109
soil_seedbank_absent                             53
canopy_seedbank_absent soil_seedbank_absent      22
canopy_seedbank soil_seedbank                     5
Name: count, dtype: int64


## Explore traits per taxon name

Example of how to query information for a single species.

In [37]:
target = ATtaxa[ATtaxa['taxon_name'] == "Actinotus helianthi"] 
target

,taxon_name,taxon_rank,taxonomic_status,taxonomic_dataset,taxon_name_alternatives,genus,family,binomial,trinomial,taxon_distribution,establishment_means,scientific_name,taxon_id,taxon_id_genus,taxon_id_family,scientific_name_id
1836,Actinotus helianthi,species,accepted,APC,NaN,Actinotus,Apiaceae,Actinotus helianthi,NaN,"Qld, NSW, Vic (naturalised)",native and naturalised,Actinotus helianthi Labill.,https://id.biodiversity.org.au/node/apni/2895645,https://id.biodiversity.org.au/taxon/apni/5173...,https://id.biodiversity.org.au/taxon/apni/5173...,https://id.biodiversity.org.au/name/apni/75081


In [38]:
qry = target['scientific_name_id'].values[0]+".json"
qry
#apniData=json.loads(qry) # error?

'https://id.biodiversity.org.au/name/apni/75081.json'

In [39]:
apniData = json.loads(getResponse(qry))
apniData.keys()

dict_keys(['class', '_links', 'audit', 'fullName', 'fullNameHtml', 'nameElement', 'simpleName', 'sortName', 'rank', 'verbatimRank', 'type', 'status', 'tags', 'family', 'parent', 'basionym', 'secondParent', 'instances', 'author', 'baseAuthor', 'exAuthor', 'exBaseAuthor', 'sanctioningAuthor', 'primaryInstance'])

In [40]:
spp_info = BIONET[BIONET['scientificName'] == "Actinotus helianthi"] 
spp_info.transpose()

,3
speciesID,2361
taxonRank,Species
classID,38
class,Flora
orderID,137
order,Flora
familyID,157
family,Apiaceae
sortOrder,6031
genusID,1823.0


In [41]:
sppname='Actinotus helianthi'
traits=('resprouting_capacity','post_fire_flowering','seedbank_location')

ss = (ATtraits['taxon_name']==sppname) & (ATtraits['trait_name'].isin(traits))
ATtraits[ss]

,dataset_id,taxon_name,observation_id,trait_name,value,unit,entity_type,value_type,basis_of_value,replicates,...,source_id,location_id,entity_context_id,plot_context_id,treatment_context_id,collection_date,measurement_remarks,method_id,method_context_id,original_name
476878,Falster_2005_2,Actinotus helianthi,5,resprouting_capacity,fire_killed,NaN,population,mode,expert_score,NaN,...,NaN,1.0,NaN,NaN,NaN,2002-09/2002-09,NaN,1.0,NaN,Actinotus helianthi
628072,Kubiak_2009,Actinotus helianthi,88,resprouting_capacity,fire_killed,NaN,population,mode,expert_score,NaN,...,NaN,2.0,8.0,NaN,NaN,1994/1999,NaN,1.0,NaN,Actinotus helianthi
628073,Kubiak_2009,Actinotus helianthi,89,resprouting_capacity,fire_killed,NaN,population,mode,expert_score,NaN,...,NaN,3.0,8.0,NaN,NaN,1994/1999,NaN,1.0,NaN,Actinotus helianthi
872818,NSWFRD_2014,Actinotus helianthi,334,resprouting_capacity,fire_killed,NaN,species,mode,expert_score,NaN,...,NaN,NaN,NaN,NaN,NaN,2002/2014,NaN,1.0,NaN,Actinotus helianthi
872819,NSWFRD_2014,Actinotus helianthi,334,seedbank_location,soil_seedbank,NaN,species,mode,expert_score,NaN,...,NaN,NaN,NaN,NaN,NaN,2002/2014,NaN,1.0,NaN,Actinotus helianthi


## That is it for now!

✅ Job done! 😎👌🔥

You can:
- go [back home](../Instructions-and-workflow.ipynb),
- continue navigating the repo on [GitHub](https://github.com/ces-unsw-edu-au/fireveg-db-exports)
- continue exploring the repo on [OSF](https://osf.io/h96q2/).
- visit the database at <http://fireecologyplants.net>